In [1]:
import bs4
from collections import defaultdict    
from bs4 import BeautifulSoup    
import requests
import pandas as pd
import pymongo

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Create a new database
db = client.clinical_trial_db

# Create a new collection and reset the collection
collection = db.bristol_myers_squibb
db.drop_collection(collection)

In [2]:
def clinicalTrialsGov(nctid):
    data = defaultdict(list)
    soup = BeautifulSoup(requests.get("https://clinicaltrials.gov/ct2/show/" + nctid + "?displayxml=true").text, "xml")
    subset = ['name','status','city','zip']
    for tag in soup.find_all(subset):
        data['ct{}'.format(tag.name.capitalize())].append(tag.get_text(strip=True))

    for key in data:
        print('{}: {}'.format(key, ', '.join(data[key])))
    return data

In [3]:
data = clinicalTrialsGov('NCT01592370')

ctName: Local Institution, Local Institution, Local Institution, Division Of Hematology & Oncology Ctr. For Health Sciences, University of Colorado Denver, Yale University School Of Medicine, Local Institution, Local Institution, Cancer Institute Of Florida, Local Institution, Local Institution, Local Institution, Indiana University Melvin & Bren Simon Cancer Center, Local Institution, University Of Kansas Medical Center, The Sidney Kimmel Comprehensive Cancer Center, Dana Farber Cancer Institute, Dana-Farber Cancer Institute, University Of Michigan Health System, Mayo Clinic, Local Institution, Nebraska Cancer Specialists, John Theurer Cancer Center, Memorial Sloan Kettering Cancer Center, Local Institution, Ohio State University, Local Institution, OHSU Center for Hematologic Malignancies, Abramson Cancer Center, Fox Chase Cancer Center, Local Institution, Local Institution, Local Institution, Local Institution, Huntsman Cancer Institute At The Univ. Of Utah, Local Institution, Local

In [4]:
df = pd.DataFrame({'name':data['ctName'],'status':data['ctStatus'],'city':data['ctCity'],'zip':data['ctZip']})
df

,name,status,city,zip
0,Local Institution,Withdrawn,Little Rock,72205
1,Local Institution,Not yet recruiting,Fresno,93701
2,Local Institution,Withdrawn,Long Beach,90813
3,Division Of Hematology & Oncology Ctr. For Hea...,"Active, not recruiting",Los Angeles,90095
4,University of Colorado Denver,Recruiting,Aurora,80045
5,Yale University School Of Medicine,Completed,New Haven,06520
6,Local Institution,Withdrawn,Gainesville,32608
7,Local Institution,Withdrawn,Miami,33136
8,Cancer Institute Of Florida,Recruiting,Orlando,32804
9,Local Institution,Withdrawn,Tampa,33612-9497


In [6]:
export_csv = df.to_csv (r'C:\Users\krystal.sanhudo\Desktop\ETLct\ct-ETL.csv', index = None, header=True)

In [ ]:
# we need a table with NCT #'s so we can assign the location the ID, this avoids having the nct# repeat for
#e every location so we can optimize the data.